## SBTi-Finance Tool for Temperature Scoring & Portfolio Coverage - Target Reporting
This notebook can be used to compute your portfolio's temperature scores and save the results for SBTi's Target Validation Team (TVT) to check their validity.

#### Setting up
Imports SBTi tools, import data providers, and load the portfolio.

In [ ]:
# Uncomment the second line and comment out the third line if you wish to use the official release.
# !pip install SBTi
!pip install git+git://github.com/OFBDABV/SBTi

In [1]:
%load_ext autoreload
%autoreload 2
import SBTi
from SBTi.data.excel import ExcelProvider
from SBTi.portfolio_aggregation import PortfolioAggregationMethod
from SBTi.portfolio_coverage_tvp import PortfolioCoverageTVP
from SBTi.temperature_score import TemperatureScore
from SBTi.interfaces import ETimeFrames, EScope
import pandas as pd

#### Download resources for Google Colab environment

In [2]:
# Download the dummy data
import urllib.request
import os

if not os.path.isdir("data"):
    os.mkdir("data")
if not os.path.isfile("data/data_provider_example.xlsx"):
    urllib.request.urlretrieve("https://github.com/OFBDABV/SBTi/raw/master/examples/data/data_provider_example.xlsx", "data/data_provider_example.xlsx")
if not os.path.isfile("data/example_portfolio.csv"):
    urllib.request.urlretrieve("https://github.com/OFBDABV/SBTi/raw/master/examples/data/example_portfolio.csv", "data/example_portfolio.csv")
if not os.path.isfile("utils.py"):
    urllib.request.urlretrieve("https://github.com/OFBDABV/SBTi/raw/master/examples/utils.py", "utils.py")
try:  # Import statement when run in Google Colabs
    from utils import collect_company_contributions, plot_grouped_statistics, anonymize, print_aggregations, \
        plot_grouped_heatmap, print_scenario_gain, print_grouped_scores
except:  # Import statement when run locally
    from examples.utils import collect_company_contributions, plot_grouped_statistics, anonymize, print_aggregations, \
        plot_grouped_heatmap, print_scenario_gain, print_grouped_scores

Specify the data files to use (portfolio and company data). Per default example data is used.
To change the files, first upload (when using Google Colab) or place the files to the data map (when running locally).
Next, change the file locations between the quotes below.

In [3]:
provider = ExcelProvider(path="data/data_provider_example.xlsx")
portfolio = pd.read_csv("data/example_portfolio.csv", encoding="iso-8859-1")
companies = SBTi.utils.dataframe_to_portfolio(portfolio)

### Compute temperature scores
Configure computation settings

You can change the default score (fallback_score) and aggregation method (aggregation_method) below. The values specified here are the recommended defaults by SBTi.
The options of the default score are 3.2, 3.9 or 4.5. Options for the aggregation method are WATS, TETS, AOTS, MOTS, EOTS, ECOTS, and ROTS.

In [4]:
time_frames = [SBTi.interfaces.ETimeFrames.MID]
scopes = [EScope.S1S2, EScope.S1S2S3]
fallback_score = 3.2
model = 4
aggregation_method = PortfolioAggregationMethod.WATS
grouping = ['sector']

In [5]:
temperature_score = TemperatureScore(
    time_frames=time_frames,
    scopes=scopes,
    fallback_score=fallback_score,
    model=model,
    aggregation_method=aggregation_method,
    grouping=grouping
)

Compute scores and aggregated scores.

In [6]:
amended_portfolio = temperature_score.calculate(data_providers=[provider], portfolio=companies)
aggregated_scores = temperature_score.aggregate_scores(amended_portfolio)

### Checks
The following scores are performed:
- Portfolio score
- Score per sector
- Percentage covered by targets vs. default scores
- Percentage of SBTs in portfolio

In [7]:
# Portfolio score
print_aggregations(aggregated_scores)

mid - S1S2: 2.55 degrees Celcius
mid - S1S2S3: 2.82 degrees Celcius


In [8]:
# Score per sector
print_grouped_scores(aggregated_scores)


mid - S1S2
Communication Services: 3.20 degrees Celcius
Consumer Discretionary: 2.00 degrees Celcius
Consumer Staples: 3.20 degrees Celcius
Energy: 3.20 degrees Celcius
Financials: 3.20 degrees Celcius
Health Care: 2.70 degrees Celcius
Industrials: 2.33 degrees Celcius
Information Technology: 1.20 degrees Celcius
Materials: 3.20 degrees Celcius
Real Estate: 2.59 degrees Celcius
Utilities: 2.44 degrees Celcius

mid - S1S2S3
Communication Services: 3.20 degrees Celcius
Consumer Discretionary: 3.02 degrees Celcius
Consumer Staples: 3.20 degrees Celcius
Energy: 3.20 degrees Celcius
Financials: 3.11 degrees Celcius
Health Care: 3.05 degrees Celcius
Industrials: 2.70 degrees Celcius
Information Technology: 2.08 degrees Celcius
Materials: 3.20 degrees Celcius
Real Estate: 3.08 degrees Celcius
Utilities: 2.50 degrees Celcius


In [9]:
# Percentage of score based on default score
amended_portfolio[['company_name', 'scope', 'company_id', 'temperature_results']]

,company_name,scope,company_id,temperature_results
0,"Advanced Micro Devices, Inc",S1S2,US0079031078,0.000000
2,"Advanced Micro Devices, Inc",S1S2S3,US0079031078,0.727326
3,Adobe Systems Inc.,S1S2,US00724F1012,0.000000
5,Adobe Systems Inc.,S1S2S3,US00724F1012,0.000000
6,Capgemini Group,S1S2,FR0000125338,1.000000
...,...,...,...,...
140,Company AD,S1S2S3,KE0000000030,1.000000
141,Company AE,S1S2,TH0000000031,1.000000
143,Company AE,S1S2S3,TH0000000031,1.000000
144,Company AF,S1S2,ID0000000032,1.000000


In [10]:
# Percentage of SBTs in portfolio
portfolio_coverage_tvp = PortfolioCoverageTVP()
coverage = portfolio_coverage_tvp.get_portfolio_coverage(amended_portfolio.copy(), PortfolioAggregationMethod.WATS)
print("Part of portfolio with SBTi approved targets is {c:.2f}%".format(c=coverage))

Part of portfolio with SBTi approved targets is 35.45%


### Save anonymized data for SBTi target validation
In order for the targets to be validated by SBTi, you can save your data locally. By running the anonymize function, you can replace company identifiers with meaningless substitutes.

In [11]:
portfolio, provider = anonymize(portfolio, provider)

In order to store the portfolio and provider data locally, two options apply:
1. You are running the SBTi tool locally or from Google Colab
2. You are running the SBTi tool from a Docker container

If you run the SBTi tool locally or from Google Colab, you:
- Specify and filenames in the cell below
- Run the cell below

In [12]:
portfolio_filename = 'portfolio.xlsx'
provider_filename = 'provider.xlsx'
portfolio.to_excel(portfolio_filename, index=False)

writer = pd.ExcelWriter(provider_filename, engine='openpyxl')
provider.data['fundamental_data'].to_excel(writer, sheet_name='fundamental_data')
provider.data['target_data'].to_excel(writer, sheet_name='target_data')
writer.save()

If you run the SBTi tool locally, you finished the notebook at this point. If you run the SBTi tool from Google Colab, you:
- Click on the files icon in the left pane
- Click the three dots that appear after hovering over the file
- Download the file to your local machine

and you finished the notebook.